In [ ]:
from pathlib import Path

import kagglehub
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

Reading the raw dataset.

In [ ]:
path = kagglehub.dataset_download("divyapatel4/microsoft-pens-personalized-news-headlines/versions/7")
df = pd.read_csv(open(Path(path, "PENS", "news.tsv")), sep="\t", encoding="utf-8")
df

Dropping useless columns.

In [ ]:
df = df.drop(columns=["News ID", "Category", "Title entity", "Entity content"])

Renaming columns.

In [ ]:
df = df.rename(columns={"Topic": "topic", "Headline": "title", "News body": "body"})

Stripping the titles and the bodies.

In [ ]:
df["title"] = (df["title"].str.strip()).replace("", np.nan)
df["body"] = (df["body"].str.strip()).replace("", np.nan)

Dropping rows missing either the topic or both the title and the body.

In [ ]:
df = df.dropna(subset=["topic"])
df = df.dropna(subset=["title", "body"], how="all")

Dropping rows with potentially politically ambiguous topics.

In [ ]:
df["topic"].unique()

In [ ]:
ambiguous_topics = [
    "ads-tax-proof",
    "binge",
    "causes",
    "causes-food-insecurity",
    "causes-homelessness",
    "causes-human-rights",
    "causes-poverty",
    "columnists",
    "comedy",
    "downtime",
    "finance-companies",
    "finance-education",
    "finance-healthcare",
    "finance-insurance",
    "finance-retirement",
    "finance-saving-investing",
    "finance-top-stocks",
    "finance-top-stories",
    "finance-video",
    "financenews",
    "healthtrending",
    "indepth",
    "lifestyleroyals",
    "localnews",
    "markets",
    "movievideo",
    "news",
    "news-causes-lgbt",
    "newslocal",
    "newsopinion",
    "newsphotos",
    "newsscienceandtechnology",
    "newstrends",
    "newsuk",
    "newsus",
    "newsvideo",
    "newsvideos",
    "newsworld",
    "northamerica-video",
    "people-places",
    "peopleandplaces",
    "seasonalvideos",
    "spotlight",
    "topnews",
    "tunedin",
    "tv",
    "tv-gallery",
    "tv-recaps",
    "tvnews",
    "viral",
    "watch",
]

df = df[~df["topic"].isin(ambiguous_topics)]

Printing body duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]

Dropping the body duplicates.

In [ ]:
df = df.drop_duplicates(subset="body")

Adding the politicalness label column.

In [ ]:
political_topics = [
    "brexit",
    "narendramodi_opinion",
    "newsfactcheck",
    "newspolitics",
    "politicsvideo",
    "royals",
]

df["politicalness"] = pd.Categorical(
    ["political" if topic in political_topics else "non-political" for topic in df["topic"]]
)

Dropping useless columns.

In [ ]:
df = df.drop(columns=["topic"])

Inspecting the title and the body length.

In [ ]:
df["title_length"] = df["title"].str.len()
df["title_word_count"] = df["title"].str.split().str.len()
df["body_length"] = df["body"].str.len()
df["body_word_count"] = df["body"].str.split().str.len()
df = df.sort_values(by="body_length")
df.head()

After the inspection, rows with bodies shorter than 4 words seem to contain no useful value. Removing them.

In [ ]:
body_word_count_lower_bound = 4
df = df[df["body_word_count"] >= body_word_count_lower_bound]
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df["body_length"].iloc[::1000], df["body_length"].tail(1)]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

In [ ]:
df["title_length"].mean()

In [ ]:
df["body_length"].mean()

The politicalness distribution.

In [ ]:
df.groupby("politicalness", observed=True).size().plot.pie(autopct="%1.1f%%");

The distribution of body length sums per politicalness.

In [ ]:
df.groupby("politicalness", observed=True)["body_length"].sum().plot.pie(autopct="%1.1f%%", ylabel="");

The body length distribution by politicalness.

In [ ]:
plt.figure(figsize=(10, 6))

for politicalness in df["politicalness"].unique():
    df_politicalness = df[df["politicalness"] == politicalness]
    downsampled = pd.concat(
        # Ensuring to include the last (longest) item.
        [df_politicalness["body_length"].iloc[::10], df_politicalness["body_length"].tail(1)]
    ).drop_duplicates().reset_index(drop=True)
    plt.plot(downsampled, label=politicalness)

plt.xlabel("downsampled index")
plt.ylabel("body length")
plt.title("body length by politicalness")
plt.legend(title="politicalness")
plt.show()

In [ ]:
df.to_parquet("../preprocessed/pens.parquet")